<a href="https://colab.research.google.com/github/Mahfujul-01726/Random/blob/main/SynergyX2024_Datathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
data.shape

(621165, 68)

In [ ]:
# Import necessary module for displaying all columns
pd.set_option('display.max_columns', None)  # Ensures all columns are displayed

# Display the first five rows
print(data.head())

   id        v1        v2       v3       v4       v5  v6     v7      v8  v9  \
0   1  36802507  39555223  10000.0  10000.0  10000.0   0  12.39  334.01   2   
1   2  34373063  37036411  20000.0  20000.0  20000.0   1  16.49  491.59   3   
2   3  39329649  42133381   8000.0   8000.0   8000.0   0   9.49  256.23   1   
3   4  54898516  58469236  16000.0  16000.0  16000.0   0  11.53  527.85   1   
4   5  42513663  45480375  10200.0  10200.0  10200.0   0  18.25  370.04   4   

   v10  v11  v12      v13  v14       v15  v16  v17  v18    v19  v20  v21  \
0   10    5    5  35000.0    1  Jan-2015  0.0    0    1  26.95  0.0  2.0   
1   17    1    1  66000.0    1  Nov-2014  0.0    0    2  24.44  1.0  0.0   
2    6    7    5  35000.0    1  Jan-2015  0.0    0    1  21.91  0.0  0.0   
3    9    7    5  70000.0    1  Jul-2015  0.0    0    1  13.99  1.0  0.0   
4   20    1    1  30000.0    0  Mar-2015  0.0    0    9  25.16  1.0  0.0   

    v22   v23   v24  v25      v26   v27   v28  v29       v30       v

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Load the dataset
data = pd.read_csv("train.csv")

# Separate features and target
X = data.drop(columns=['v16'], errors='ignore')  # Feature matrix without target column
y = data['v16'] if 'v16' in data.columns else None  # Target variable

# Drop non-numeric columns from features, assuming 'id' is retained separately for later merging
X_numeric = X.select_dtypes(include=[int, float])

# Impute missing values in the feature matrix using the mean
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_numeric)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Apply PCA to retain 95% of the variance
pca = PCA(0.95)
X_pca = pca.fit_transform(X_scaled)

# Create a DataFrame with the principal components
pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
X_pca_df = pd.DataFrame(data=X_pca, columns=pca_columns)

# Combine with the original 'id' column if it exists
if 'id' in data.columns:
    final_df = pd.concat([data[['id']].reset_index(drop=True), X_pca_df], axis=1)
else:
    print("Warning: 'id' column not found in the original data. Proceeding without it.")
    final_df = X_pca_df

# Add the target variable back if it exists
if y is not None:
    final_df = pd.concat([final_df, y.reset_index(drop=True).rename('v16')], axis=1)

# Output the results
print(final_df)


            id       PC1       PC2       PC3       PC4       PC5       PC6  \
0            1 -1.901848 -0.321198  0.479051  0.864787 -0.177168 -2.011804   
1            2  1.460722  1.422081  0.859594  1.794057 -0.304390 -0.151057   
2            3 -2.766509 -0.192471 -0.276345 -0.170507 -0.313756 -2.491909   
3            4 -1.134767  2.205174 -0.098289 -0.015066  0.162879  1.183892   
4            5 -1.368902  0.387452  1.282945  2.105966 -0.130027  0.274153   
...        ...       ...       ...       ...       ...       ...       ...   
621160  621161 -3.361996  0.045208 -0.557432 -0.990810 -0.026975  0.621957   
621161  621162  1.193795 -0.651763 -0.441369 -0.490231 -0.004835 -0.035984   
621162  621163  1.500297 -2.969715 -0.757388 -0.604006 -0.277567 -1.796209   
621163  621164 -0.374131  2.167216  0.527043  1.214250 -0.267335 -1.360255   
621164  621165  0.101524 -1.779456 -0.778518 -1.416086  0.102181 -1.068241   

             PC7       PC8       PC9      PC10      PC11      P

In [ ]:
# Count NaN values in the target column 'v16'
nan_count = final_df['v16'].isna().sum()

print(f"Number of NaN values in 'v16': {nan_count}")


Number of NaN values in 'v16': 184700


In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Load the dataset
data = pd.read_csv("train.csv")

# Separate features and target
X = data.drop(columns=['v16'], errors='ignore')  # Feature matrix without target column
y = data['v16'] if 'v16' in data.columns else None  # Target variable

# Drop non-numeric columns from features, assuming 'id' is retained separately for later merging
X_numeric = X.select_dtypes(include=[int, float])

# Impute missing values in the feature matrix using the mean
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_numeric)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Apply PCA to retain 95% of the variance
pca = PCA(0.95)
X_pca = pca.fit_transform(X_scaled)

# Create a DataFrame with the principal components
pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
X_pca_df = pd.DataFrame(data=X_pca, columns=pca_columns)

# Combine with the original 'id' column if it exists
if 'id' in data.columns:
    final_df = pd.concat([data[['id']].reset_index(drop=True), X_pca_df], axis=1)
else:
    print("Warning: 'id' column not found in the original data. Proceeding without it.")
    final_df = X_pca_df

# Handle missing values in the target variable 'v16' using imputation
if y is not None:
    target_imputer = SimpleImputer(strategy='most_frequent')  # Impute with the most frequent class for classification
    y_imputed = target_imputer.fit_transform(y.values.reshape(-1, 1))  # Reshaping for imputation

    # Add the imputed target variable back to the final DataFrame
    final_df = pd.concat([final_df, pd.Series(y_imputed.flatten(), name='v16').reset_index(drop=True)], axis=1)

# Output the results
print(final_df)


            id       PC1       PC2       PC3       PC4       PC5       PC6  \
0            1 -1.901848 -0.321198  0.479051  0.864787 -0.177168 -2.011804   
1            2  1.460722  1.422081  0.859594  1.794057 -0.304390 -0.151057   
2            3 -2.766509 -0.192471 -0.276345 -0.170507 -0.313756 -2.491909   
3            4 -1.134767  2.205174 -0.098289 -0.015066  0.162879  1.183892   
4            5 -1.368902  0.387452  1.282945  2.105966 -0.130027  0.274153   
...        ...       ...       ...       ...       ...       ...       ...   
621160  621161 -3.361996  0.045208 -0.557432 -0.990810 -0.026975  0.621957   
621161  621162  1.193795 -0.651763 -0.441369 -0.490231 -0.004835 -0.035984   
621162  621163  1.500297 -2.969715 -0.757388 -0.604006 -0.277567 -1.796209   
621163  621164 -0.374131  2.167216  0.527043  1.214250 -0.267335 -1.360255   
621164  621165  0.101524 -1.779456 -0.778518 -1.416086  0.102181 -1.068241   

             PC7       PC8       PC9      PC10      PC11      P

In [ ]:
# Count NaN values in the target column 'v16'
nan_count = final_df['v16'].isna().sum()

print(f"Number of NaN values in 'v16': {nan_count}")

Number of NaN values in 'v16': 0


In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
data = pd.read_csv("train.csv")

# Separate features and target
X = data.drop(columns=['v16'], errors='ignore')  # Feature matrix without target column
y = data['v16'] if 'v16' in data.columns else None  # Target variable

# Identify non-numeric columns
non_numeric_cols = X.select_dtypes(exclude=[int, float]).columns
print("Non-numeric columns:", non_numeric_cols)

# Option 1: Apply Label Encoding (for ordinal data)
label_encoder = LabelEncoder()
for col in non_numeric_cols:
    X[col] = label_encoder.fit_transform(X[col])

# Alternatively, Option 2: Apply One-Hot Encoding (for nominal data)
# X = pd.get_dummies(X, columns=non_numeric_cols, drop_first=True)

# Drop non-numeric columns from features (now they are encoded)
X_numeric = X.select_dtypes(include=[int, float])

# Impute missing values in the feature matrix using the mean
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_numeric)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Apply PCA to retain 95% of the variance
pca = PCA(0.95)
X_pca = pca.fit_transform(X_scaled)

# Create a DataFrame with the principal components
pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
X_pca_df = pd.DataFrame(data=X_pca, columns=pca_columns)

# Combine with the original 'id' column if it exists
if 'id' in data.columns:
    final_df = pd.concat([data[['id']].reset_index(drop=True), X_pca_df], axis=1)
else:
    print("Warning: 'id' column not found in the original data. Proceeding without it.")
    final_df = X_pca_df

# Impute missing values in the target variable using the mean
if y is not None:
    y_imputer = SimpleImputer(strategy='mean')
    y_imputed = y_imputer.fit_transform(y.values.reshape(-1, 1))  # Reshape to 2D array
    y_imputed = y_imputed.flatten()  # Flatten back to 1D

    # Add the imputed target variable back to the final DataFrame
    final_df = pd.concat([final_df, pd.Series(y_imputed, name='v16').reset_index(drop=True)], axis=1)

print(final_df)

Non-numeric columns: Index(['v15', 'v39', 'v41', 'v42'], dtype='object')
            id       PC1       PC2       PC3       PC4       PC5       PC6  \
0            1 -1.917209 -0.148872  0.516312  0.900320 -0.179700 -2.005388   
1            2  1.331313  1.845028  0.879955  1.764853 -0.310367 -0.228063   
2            3 -2.782442 -0.114113 -0.260353 -0.124727 -0.317031 -2.491713   
3            4 -1.265401  2.236560 -0.135987 -0.078908  0.164585  1.105585   
4            5 -1.428050  0.589388  1.320871  2.083077 -0.129150  0.269370   
...        ...       ...       ...       ...       ...       ...       ...   
621160  621161 -3.399161  0.079942 -0.564869 -0.956519 -0.028773  0.527453   
621161  621162  1.173444 -0.203945 -0.408814 -0.393082 -0.015176 -0.226992   
621162  621163  1.591787 -2.432676 -0.673505 -0.405207 -0.286932 -2.022114   
621163  621164 -0.503059  2.272436  0.511084  1.141254 -0.271821 -1.327706   
621164  621165  0.151674 -1.463542 -0.742910 -1.281475  0.096628 -1.1

In [ ]:
# Count NaN values in the target column 'v16'
nan_count = final_df['v16'].isna().sum()

print(f"Number of NaN values in 'v16': {nan_count}")

Number of NaN values in 'v16': 0


In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor  # Use RandomForestRegressor for regression tasks
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
data = pd.read_csv("train.csv")

# Separate features and target
X = data.drop(columns=['v16'], errors='ignore')  # Feature matrix without target column
y = data['v16'] if 'v16' in data.columns else None  # Target variable

# Identify non-numeric columns
non_numeric_cols = X.select_dtypes(exclude=[int, float]).columns
print("Non-numeric columns:", non_numeric_cols)

# Option 1: Apply Label Encoding (for ordinal data)
label_encoder = LabelEncoder()
for col in non_numeric_cols:
    X[col] = label_encoder.fit_transform(X[col])

# Alternatively, Option 2: Apply One-Hot Encoding (for nominal data)
# X = pd.get_dummies(X, columns=non_numeric_cols, drop_first=True)

# Drop non-numeric columns from features (now they are encoded)
X_numeric = X.select_dtypes(include=[int, float])

# Impute missing values in the feature matrix using the mean
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_numeric)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Apply PCA to retain 95% of the variance
pca = PCA(0.95)
X_pca = pca.fit_transform(X_scaled)

# Create a DataFrame with the principal components
pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
X_pca_df = pd.DataFrame(data=X_pca, columns=pca_columns)

# Combine with the original 'id' column if it exists
if 'id' in data.columns:
    final_df = pd.concat([data[['id']].reset_index(drop=True), X_pca_df], axis=1)
else:
    print("Warning: 'id' column not found in the original data. Proceeding without it.")
    final_df = X_pca_df

# Impute missing values in the target variable using the mean
if y is not None:
    y_imputer = SimpleImputer(strategy='mean')
    y_imputed = y_imputer.fit_transform(y.values.reshape(-1, 1))  # Reshape to 2D array
    y_imputed = y_imputed.flatten()  # Flatten back to 1D

    # Add the imputed target variable back to the final DataFrame
    final_df = pd.concat([final_df, pd.Series(y_imputed, name='v16').reset_index(drop=True)], axis=1)

print(final_df)

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(final_df.drop(columns=['v16']), final_df['v16'], test_size=0.2, random_state=42)

# Initialize the Random Forest model (Regressor for regression tasks)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)  # Use RandomForestRegressor for regression tasks

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
print("Mean Squared Error on test set:", mean_squared_error(y_test, y_pred))
print("R^2 Score on test set:", r2_score(y_test, y_pred))


Non-numeric columns: Index(['v15', 'v39', 'v41', 'v42'], dtype='object')
            id       PC1       PC2       PC3       PC4       PC5       PC6  \
0            1 -1.862912 -0.087956  0.804218 -0.765478 -0.182189 -1.715079   
1            2  1.355553  1.912952  1.269934 -1.443751 -0.294843 -0.306946   
2            3 -2.756326 -0.108669 -0.246802  0.027452 -0.296281 -2.285973   
3            4 -1.291306  2.203269 -0.205199  0.104688  0.205529  0.658562   
4            5 -1.385379  0.642770  1.848127 -1.675149 -0.131546  0.286638   
...        ...       ...       ...       ...       ...       ...       ...   
241241  241242 -3.211634  1.326530 -1.006782  0.944360 -0.092585 -0.250760   
241242  241243  0.649629  4.142882 -1.458122  0.891040 -0.196449 -1.246934   
241243  241244 -0.970156  0.619281  0.651801 -1.006273 -0.222309 -2.221044   
241244  241245 -1.775040  0.702773 -0.826489  1.262844  0.227383  1.564395   
241245  241246  1.169239  1.738705  0.204711  0.468227  0.447051  2.9

KeyboardInterrupt: 

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier  # Use RandomForestRegressor for regression tasks
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report  # For classification tasks
from sklearn.impute import SimpleImputer  # For imputing missing target values

# Load the dataset
data = pd.read_csv("train.csv")

# Separate features and target
X = data.drop(columns=['v16'], errors='ignore')  # Feature matrix without target column
y = data['v16'] if 'v16' in data.columns else None  # Target variable

# Drop non-numeric columns from features, assuming 'id' is retained separately for later merging
X_numeric = X.select_dtypes(include=[int, float])

# Impute missing values in the feature matrix using the mean (for numeric features)
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_numeric)

# Initialize the SimpleImputer for the target variable 'v16' (impute with the most frequent class)
y_imputer = SimpleImputer(strategy='most_frequent')  # For classification, you can also use 'mean' for regression
y_imputed = y_imputer.fit_transform(data[['v16']])

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_imputed.ravel(), test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier (use RandomForestRegressor if it's a regression task)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # Adjust parameters as needed

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model (for classification)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

# If it's a regression problem, you can use the following metrics instead:
# from sklearn.metrics import mean_squared_error, r2_score
# print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
# print("R^2 Score:", r2_score(y_test, y_pred))


Accuracy: 0.9772685196364895
Classification Report:
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99    121237
         1.0       0.89      0.07      0.12      2996

    accuracy                           0.98    124233
   macro avg       0.93      0.53      0.56    124233
weighted avg       0.98      0.98      0.97    124233



# Suppose it is final code

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier  # Use RandomForestRegressor if regression task
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report  # For classification tasks
# from sklearn.metrics import mean_squared_error, r2_score  # Uncomment for regression tasks

# Load the dataset
data = pd.read_csv("train.csv")

# Separate features and target
X = data.drop(columns=['v16'], errors='ignore')  # Feature matrix without target column
y = data['v16'] if 'v16' in data.columns else None  # Target variable

# Drop non-numeric columns from features, assuming 'id' is retained separately for later merging
X_numeric = X.select_dtypes(include=[int, float])

# Impute missing values in the feature matrix using the mean
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_numeric)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Apply PCA to retain 95% of the variance
pca = PCA(0.95)
X_pca = pca.fit_transform(X_scaled)

# Create a DataFrame with the principal components
pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
X_pca_df = pd.DataFrame(data=X_pca, columns=pca_columns)

# Combine with the original 'id' column if it exists
if 'id' in data.columns:
    final_df = pd.concat([data[['id']].reset_index(drop=True), X_pca_df], axis=1)
else:
    print("Warning: 'id' column not found in the original data. Proceeding without it.")
    final_df = X_pca_df

# Handle missing values in the target variable 'v16' using imputation
if y is not None:
    target_imputer = SimpleImputer(strategy='most_frequent')  # Impute with the most frequent class for classification
    y_imputed = target_imputer.fit_transform(y.values.reshape(-1, 1))  # Reshaping for imputation

    # Add the imputed target variable back to the final DataFrame
    final_df = pd.concat([final_df, pd.Series(y_imputed.flatten(), name='v16').reset_index(drop=True)], axis=1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(final_df.drop(columns=['v16'], errors='ignore'),
                                                    final_df['v16'],
                                                    test_size=0.2,
                                                    random_state=42)

# Initialize the Random Forest Classifier (use RandomForestRegressor for regression tasks)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust the parameters

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model (for classification)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

# If it's a regression problem, you can use the following metrics instead:
# from sklearn.metrics import mean_squared_error, r2_score
# print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
# print("R^2 Score:", r2_score(y_test, y_pred))


Accuracy: 0.9758759749824926
Classification Report:
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99    121237
         1.0       0.47      0.00      0.01      2996

    accuracy                           0.98    124233
   macro avg       0.72      0.50      0.50    124233
weighted avg       0.96      0.98      0.96    124233



# Test data set ,,, now it intigrate with train data set

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Load the dataset
data = pd.read_csv("test.csv")

# Separate the 'id' column if it exists, to retain it for later merging
id_column = data[['id']] if 'id' in data.columns else None
X = data.drop(columns=['id'], errors='ignore')  # Drop 'id' if present to work on features only

# Identify non-numeric columns
non_numeric_cols = X.select_dtypes(exclude=[int, float]).columns
print("Non-numeric columns:", non_numeric_cols.tolist())

# Option 1: Apply Label Encoding (suitable for ordinal data)
label_encoder = LabelEncoder()
for col in non_numeric_cols:
    X[col] = label_encoder.fit_transform(X[col].astype(str))  # Convert to string if needed for encoding

# Alternatively, Option 2: Apply One-Hot Encoding (suitable for nominal data)
# X = pd.get_dummies(X, columns=non_numeric_cols, drop_first=True)

# Drop non-numeric columns from features (they are now encoded)
X_numeric = X.select_dtypes(include=[int, float])

# Impute missing values in the numeric feature matrix using the mean
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_numeric)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Apply PCA to retain 95% of the variance
pca = PCA(0.95)
X_pca = pca.fit_transform(X_scaled)

# Create a DataFrame with the principal components
pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
X_pca_df = pd.DataFrame(data=X_pca, columns=pca_columns)

# Combine with the original 'id' column if it exists
if id_column is not None:
    final_df1 = pd.concat([id_column.reset_index(drop=True), X_pca_df], axis=1)
else:
    final_df1 = X_pca_df

# Output the results
print(final_df1)


Non-numeric columns: ['v15', 'v39', 'v41', 'v42']
            id       PC1       PC2       PC3       PC4       PC5       PC6  \
0            1  1.433996 -0.947036 -1.070934 -0.809316 -0.295062 -1.955135   
1            2 -2.034131 -1.780225  0.872397  0.873145 -0.005524 -0.061383   
2            3  5.345284  2.458769  0.660579  1.147009 -0.091804 -2.392109   
3            4  1.811530  3.294641  0.026159  0.299698 -0.152482 -1.023236   
4            5  3.924497  0.776283  0.066962  0.265043  0.053768 -0.338255   
...        ...       ...       ...       ...       ...       ...       ...   
186729  186730 -0.707817 -1.394646  1.500056  1.445444  0.164144  0.536443   
186730  186731 -0.336867 -2.437583  0.478545  0.487023  0.100110  0.048660   
186731  186732  0.175935  0.096695 -1.514872 -1.816959  0.160420 -0.108661   
186732  186733 -0.018771  2.530902  0.901137  0.950029  0.050608 -0.313806   
186733  186734  2.832304  1.779116  3.463008  3.628876  0.277030  1.093365   

             

# Final code after integratiion

In [ ]:
final_df.shape

(621165, 43)

# Test Data set

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer  # Import SimpleImputer

# Load the dataset
data = pd.read_csv("test.csv")

# Drop non-numeric columns, assuming 'id' is retained separately for later merging
data_numeric = data.select_dtypes(include=[int, float])

# Impute missing values using the mean
imputer = SimpleImputer(strategy='mean')  # Create an imputer instance
data_imputed = imputer.fit_transform(data_numeric)  # Fit and transform the data

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_imputed)  # Use imputed data for scaling

# Apply PCA to retain 95% variance
pca = PCA(0.95)
principal_components = pca.fit_transform(scaled_data)

# Create a DataFrame with the principal components
# Ensure that the number of columns in 'principal_components' matches dynamically with the PCA output
pca_columns = [f'PC{i+1}' for i in range(principal_components.shape[1])]
pca_df = pd.DataFrame(data=principal_components, columns=pca_columns)

# Combine with the original 'id' column
# Make sure 'id' exists in the original data
if 'id' in data.columns:
    final_df = pd.concat([data[['id']].reset_index(drop=True), pca_df], axis=1)
else:
    print("Warning: 'id' column not found in the original data. Proceeding without it.")
    final_df = pca_df

# Output the results
print(final_df)


            id       PC1       PC2       PC3       PC4       PC5       PC6  \
0            1  1.447940 -1.031742 -1.086963 -0.830999 -0.292706 -1.942642   
1            2 -2.021247 -1.870304  0.845322  0.860733 -0.003240 -0.048324   
2            3  5.336411  2.519909  0.668718  1.170826 -0.087026 -2.407870   
3            4  1.829537  3.228858  0.027728  0.304632 -0.150098 -1.015597   
4            5  3.922917  0.785828  0.064960  0.270616  0.057746 -0.344591   
...        ...       ...       ...       ...       ...       ...       ...   
186729  186730 -0.723745 -1.337983  1.517132  1.474750  0.160323  0.546402   
186730  186731 -0.371750 -2.285737  0.505051  0.518021  0.097893  0.043259   
186731  186732  0.190465  0.036798 -1.509402 -1.835396  0.157658 -0.093349   
186732  186733  0.000846  2.458180  0.904829  0.956019  0.047638 -0.298822   
186733  186734  2.839280  1.748881  3.455433  3.654427  0.275282  1.096375   

             PC7       PC8       PC9  ...      PC31      PC32  

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
data = pd.read_csv("train.csv")

# Drop non-numeric columns except for the target if it's not numeric
data_numeric = data.select_dtypes(include=[int, float])

# Impute missing values using the mean
imputer = SimpleImputer(strategy='mean')
data_imputed = imputer.fit_transform(data_numeric)

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_imputed)

# Apply PCA to retain 95% variance
pca = PCA(0.95)
principal_components = pca.fit_transform(scaled_data)

# Create a DataFrame with the principal components
pca_columns = [f'PC{i+1}' for i in range(principal_components.shape[1])]
pca_df = pd.DataFrame(data=principal_components, columns=pca_columns)

# Check and combine with 'id' column if it exists
if 'id' in data.columns:
    final_df = pd.concat([data[['id']].reset_index(drop=True), pca_df], axis=1)
else:
    print("Warning: 'id' column not found in the original data. Proceeding without it.")
    final_df = pca_df

# Use 'v17' as the target column
target_column_name = 'PC16'
if target_column_name in data.columns:
    X = pca_df  # Use the PCA-transformed data as features
    y = data[target_column_name]  # Extract the target column from the original data
else:
    raise ValueError(f"The dataset doesn't contain a '{target_column_name}' column.")

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Random Forest Classifier Accuracy: {accuracy:.4f}')


ValueError: The dataset doesn't contain a 'PC16' column.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load the dataset
data = pd.read_csv("modified_train.csv")

# Ensure all necessary columns are present
if 'PC16' not in data.columns:
    raise ValueError("The dataset doesn't contain a 'PC16' column.")

# Prepare the features and target
X = data.drop(columns=['id', 'PC16'])
y = data['PC16']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.4f}')


In [1]:
import pandas as pd

# Load the datasets
df1 = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')

# Merge the datasets on a common column 'id'
merged_df = pd.merge(df1, df2, on='id', how='inner')

# View the merged data
merged_df.head()


,id,v1_x,v2_x,v3_x,v4_x,v5_x,v6_x,v7_x,v8_x,v9_x,...,v58_y,v59_y,v60_y,v61_y,v62_y,v63_y,v64_y,v65_y,v66_y,v67_y
0,1,36802507,39555223,10000.0,10000.0,10000.0,0,12.39,334.01,2,...,NaN,NaN,NaN,NaN,NaN,NaN,14900.0,NaN,NaN,NaN
1,2,34373063,37036411,20000.0,20000.0,20000.0,1,16.49,491.59,3,...,NaN,NaN,NaN,NaN,NaN,NaN,15500.0,NaN,NaN,NaN
2,3,39329649,42133381,8000.0,8000.0,8000.0,0,9.49,256.23,1,...,NaN,NaN,NaN,NaN,NaN,NaN,53600.0,NaN,NaN,NaN
3,4,54898516,58469236,16000.0,16000.0,16000.0,0,11.53,527.85,1,...,NaN,NaN,NaN,NaN,NaN,NaN,34800.0,NaN,NaN,NaN
4,5,42513663,45480375,10200.0,10200.0,10200.0,0,18.25,370.04,4,...,NaN,NaN,NaN,NaN,NaN,NaN,66000.0,NaN,NaN,NaN


In [3]:
df1.head()

,id,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67
0,1,36802507,39555223,10000.0,10000.0,10000.0,0,12.39,334.01,2,...,NaN,NaN,NaN,NaN,NaN,NaN,8400.0,NaN,NaN,NaN
1,2,34373063,37036411,20000.0,20000.0,20000.0,1,16.49,491.59,3,...,NaN,NaN,NaN,NaN,NaN,NaN,12100.0,NaN,NaN,NaN
2,3,39329649,42133381,8000.0,8000.0,8000.0,0,9.49,256.23,1,...,NaN,NaN,NaN,NaN,NaN,NaN,16000.0,NaN,NaN,NaN
3,4,54898516,58469236,16000.0,16000.0,16000.0,0,11.53,527.85,1,...,NaN,NaN,NaN,NaN,NaN,NaN,8300.0,NaN,NaN,NaN
4,5,42513663,45480375,10200.0,10200.0,10200.0,0,18.25,370.04,4,...,NaN,NaN,NaN,NaN,NaN,NaN,19600.0,NaN,NaN,NaN


In [4]:
df2.head()

,id,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67
0,1,27610673,30113747,21000.0,21000.0,21000.0,0,10.15,679.10,1,...,NaN,NaN,NaN,NaN,NaN,NaN,14900.0,NaN,NaN,NaN
1,2,28102260,30615399,8000.0,8000.0,8000.0,0,15.61,279.72,3,...,NaN,NaN,NaN,NaN,NaN,NaN,15500.0,NaN,NaN,NaN
2,3,57324697,61027428,35000.0,35000.0,34875.0,0,16.99,1247.68,3,...,NaN,NaN,NaN,NaN,NaN,NaN,53600.0,NaN,NaN,NaN
3,4,61402817,65521655,24000.0,24000.0,24000.0,0,12.29,800.48,2,...,NaN,NaN,NaN,NaN,NaN,NaN,34800.0,NaN,NaN,NaN
4,5,38700393,41485242,28000.0,28000.0,27950.0,1,14.31,656.02,2,...,NaN,NaN,NaN,NaN,NaN,NaN,66000.0,NaN,NaN,NaN


In [6]:
import pandas as pd
from google.colab import drive


# Load the datasets
df1 = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')

# Define the columns to merge on, excluding 'v16'
merge_columns = ['id'] + [f'v{i}' for i in range(1, 68) if i != 16]  # Excludes 'v16'

# Merge the datasets on the specified columns
merged_df = pd.merge(df1, df2, on=merge_columns, how='inner')  # Adjust 'how' as needed (inner, outer, left, right)

# View the merged data
merged_df.head()


,id,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67


In [7]:
import pandas as pd

# Load the datasets
df1 = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')

# Define the columns to merge on, excluding 'v16'
merge_columns = ['id'] + [f'v{i}' for i in range(1, 68) if i != 16]  # Excludes 'v16'

# Step 1: Merge DataFrames based on specified columns (merging columns)
merged_columns_df = pd.merge(df1, df2, on=merge_columns, how='inner')  # Adjust 'how' as needed (inner, outer, left, right)

# Step 2: Concatenate rows (row-wise merge)
merged_rows_columns_df = pd.concat([merged_columns_df, df1, df2], axis=0, ignore_index=True).drop_duplicates()

# View the final merged DataFrame
merged_rows_columns_df.head()


,id,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67
0,1,36802507,39555223,10000.0,10000.0,10000.0,0,12.39,334.01,2,...,NaN,NaN,NaN,NaN,NaN,NaN,8400.0,NaN,NaN,NaN
1,2,34373063,37036411,20000.0,20000.0,20000.0,1,16.49,491.59,3,...,NaN,NaN,NaN,NaN,NaN,NaN,12100.0,NaN,NaN,NaN
2,3,39329649,42133381,8000.0,8000.0,8000.0,0,9.49,256.23,1,...,NaN,NaN,NaN,NaN,NaN,NaN,16000.0,NaN,NaN,NaN
3,4,54898516,58469236,16000.0,16000.0,16000.0,0,11.53,527.85,1,...,NaN,NaN,NaN,NaN,NaN,NaN,8300.0,NaN,NaN,NaN
4,5,42513663,45480375,10200.0,10200.0,10200.0,0,18.25,370.04,4,...,NaN,NaN,NaN,NaN,NaN,NaN,19600.0,NaN,NaN,NaN


In [8]:
merged_rows_columns_df.shape

(807899, 68)

In [9]:
print(merged_rows_columns_df)

            id        v1        v2       v3       v4       v5  v6     v7  \
0            1  36802507  39555223  10000.0  10000.0  10000.0   0  12.39   
1            2  34373063  37036411  20000.0  20000.0  20000.0   1  16.49   
2            3  39329649  42133381   8000.0   8000.0   8000.0   0   9.49   
3            4  54898516  58469236  16000.0  16000.0  16000.0   0  11.53   
4            5  42513663  45480375  10200.0  10200.0  10200.0   0  18.25   
...        ...       ...       ...      ...      ...      ...  ..    ...   
807894  186730  22131285  24454239  10800.0  10800.0  10800.0   1  16.99   
807895  186731  14570817  16633101  10725.0  10725.0  10725.0   0  17.57   
807896  186732  34413316  37076669  16000.0  16000.0  16000.0   0   8.19   
807897  186733  64997380  69524101  18000.0  18000.0  18000.0   1  12.59   
807898  186734  56082809  59744686  23750.0  23750.0  23750.0   1  26.77   

            v8  v9  ...  v58  v59  v60  v61  v62 v63      v64  v65  v66  v67  
0       

In [11]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor  # Use RandomForestRegressor for regression tasks
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
data = merged_rows_columns_df

# Separate features and target
X = data.drop(columns=['v16'], errors='ignore')  # Feature matrix without target column
y = data['v16'] if 'v16' in data.columns else None  # Target variable

# Identify non-numeric columns
non_numeric_cols = X.select_dtypes(exclude=[int, float]).columns
print("Non-numeric columns:", non_numeric_cols)

# Option 1: Apply Label Encoding (for ordinal data)
label_encoder = LabelEncoder()
for col in non_numeric_cols:
    X[col] = label_encoder.fit_transform(X[col])

# Alternatively, Option 2: Apply One-Hot Encoding (for nominal data)
# X = pd.get_dummies(X, columns=non_numeric_cols, drop_first=True)

# Drop non-numeric columns from features (now they are encoded)
X_numeric = X.select_dtypes(include=[int, float])

# Impute missing values in the feature matrix using the mean
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_numeric)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Apply PCA to retain 95% of the variance
pca = PCA(0.95)
X_pca = pca.fit_transform(X_scaled)

# Create a DataFrame with the principal components
pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
X_pca_df = pd.DataFrame(data=X_pca, columns=pca_columns)

# Combine with the original 'id' column if it exists
if 'id' in data.columns:
    final_df = pd.concat([data[['id']].reset_index(drop=True), X_pca_df], axis=1)
else:
    print("Warning: 'id' column not found in the original data. Proceeding without it.")
    final_df = X_pca_df

# Impute missing values in the target variable using the mean
if y is not None:
    y_imputer = SimpleImputer(strategy='mean')
    y_imputed = y_imputer.fit_transform(y.values.reshape(-1, 1))  # Reshape to 2D array
    y_imputed = y_imputed.flatten()  # Flatten back to 1D

    # Add the imputed target variable back to the final DataFrame
    final_df = pd.concat([final_df, pd.Series(y_imputed, name='v16').reset_index(drop=True)], axis=1)

print(final_df)

Non-numeric columns: Index(['v15', 'v39', 'v41', 'v42'], dtype='object')
            id       PC1       PC2       PC3       PC4       PC5       PC6  \
0            1 -1.906399 -0.230438  0.554499  0.864747 -0.163991 -2.054459   
1            2  1.441092  1.595874  0.971041  1.719461 -0.306974 -0.222587   
2            3 -2.778585 -0.133561 -0.266440 -0.115158 -0.303129 -2.555043   
3            4 -1.163664  2.118228 -0.133977 -0.072008  0.130287  1.138558   
4            5 -1.408326  0.480652  1.428213  2.021121 -0.133214  0.299666   
...        ...       ...       ...       ...       ...       ...       ...   
807894  186730 -0.673428  0.183702  1.538047  1.909149  0.195424  0.249856   
807895  186731 -0.332331 -0.665859  0.753908  1.037999  0.092318 -0.101186   
807896  186732 -0.047101  0.896069 -1.174028 -1.820856  0.158890 -0.126321   
807897  186733 -0.107809  3.023788  0.514548  0.650291  0.061750 -0.261612   
807898  186734  3.026031  3.281989  2.683374  3.515061  0.323666  0.8